In [1]:
from voice import *

C:\Users\shiha\anaconda3\envs\lang\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [41]:
import moviepy.editor as mpy

def text2audio(text:str, lang:str, target = "./cache/audio.wav"):
    #audio from text
    text = text.replace('\r', ' ').replace('\n', ' ')
    o3, o4 = tts_fn(text, lang = lang)
    from scipy.io.wavfile import write
    write(target,o4[0],o4[1])

def text2video(text:str, lang:str, target = "./cache/audio.wav", title = "      "):
    #make audio
    text2audio(text, lang = lang, target = target);
    audio = mpy.AudioFileClip(target)
    #make text
    if lang == "cn":
        txt_clip  = mpy.TextClip(text, fontsize = 20, color = "white", font = "Microsoft-YaHei-&-Microsoft-YaHei-UI")
    else:
        txt_clip  = mpy.TextClip(text, fontsize = 20, color = "white")
    txt_clip = txt_clip.set_pos('bottom').set_duration(audio.duration)
    title_clip  = mpy.TextClip(title, fontsize = 40, color = "black")
    title_clip = title_clip.set_pos('top').set_duration(audio.duration)
    #make video
    clip = mpy.ImageClip("./cache/img.png")
    clip = clip.set_duration(audio.duration)
    clip.audio = audio
    clip  = mpy.CompositeVideoClip([clip, txt_clip, title_clip])
    clip.fps = 12
    #clip.write_videofile("video.mp4");
    return(clip)

def json2video(json:list, lang = "cn", output = "./cache/video.mp4"):
    videos = []
    for l in json:
        if l['cg'] != "":
            #get cg
            import wget
            wget.download(l['cg'], out="./cache/img.png")
            #split text
            text = l['summary']
            split = text.replace('。','.').split('.')
            clips = []
            for s in split:
                if len(s) > 5:
                    clips.append(text2video(text = s, lang = lang, title = l['stage']))
            videos.append(mpy.concatenate_videoclips(clips, method="compose"))
    concat_clip = mpy.concatenate_videoclips(videos, method="compose")
    concat_clip.write_videofile(output)

#output chapter content to html
def run(query:str, output = "./cache/video.mp4", path="../db/cndb"):
    import chromadb
    client = chromadb.PersistentClient(path=path)
    db = client.get_collection("langchain")
    data = db.get()
    tmp = []
    for k in range(len(data["ids"])):
        if query in data["metadatas"][k]["stage"]:
            this = {"stage": data["metadatas"][k]["stage"], 
                    "summary": data["documents"][k],
                    "cg": data["metadatas"][k]["cg"],
                   }
            tmp.append(this);
    json2video(json = tmp)



In [42]:
run("13-22")

100% [............................................................................] 606386 / 606386Moviepy - Building video ./cache/video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./cache/video.mp4



Moviepy - Done !
Moviepy - video ready ./cache/video.mp4


In [14]:
import chromadb
query = "13-22"
path="../db/cndb"
client = chromadb.PersistentClient(path=path)
db = client.get_collection("langchain")
data = db.get()
tmp = []
for k in range(len(data["ids"])):
    if query in data["metadatas"][k]["stage"]:
        this = {"stage": data["metadatas"][k]["stage"], 
                "summary": data["documents"][k],
                "cg": data["metadatas"][k]["cg"],
               }
        tmp.append(this);



In [34]:
text = tmp[0]['summary']
text.split('。')[0]

'故事的第一部分发生在一个帐篷内，有两个人物在对话'

In [38]:
clip = text2video(text.split('。')[0],lang='cn',title=tmp[0]['stage'])
clip.write_videofile("video.mp4");

Moviepy - Building video video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
